In [49]:
# Long Covid NLP from Twitter

In [94]:
import re
import string
import numpy as np 
import random
import pandas as pd 
import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib.dates as dates
from matplotlib import rc

from collections import Counter
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

from pathlib import Path
from pylab import rcParams

%matplotlib inline
%config InlineBackend.figure_format='retina'

plt.style.use('seaborn')
#plt.rcParams["figure.figsize"] = (15,10)
rcParams['figure.figsize'] = 12, 8
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

from collections import defaultdict
from textwrap import wrap
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

In [101]:
import nltk
nltk.download('stopwords') 
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from tqdm import tqdm
import os
import random
import warnings
warnings.filterwarnings("ignore")

import os

stop_words = set(stopwords.words('english'))
# View a few words from the set
list(stop_words)[0:10]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Hafidz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['does', 'how', "should've", 'up', 'ours', 'by', 'her', 've', 'hasn', 'm']

In [102]:
#helper Function which generates random colors which can be used to give different colors to your plots
def random_colours(number_of_colors):
    '''
    Simple function for random colours generation.
    Input:
        number_of_colors - integer value indicating the number of colours which are going to be generated.
    Output:
        Color in the following format: ['#E86DA4'] .
    '''
    colors = []
    for i in range(number_of_colors):
        colors.append("#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)]))
    return colors

In [22]:
!{sys.executable} -m pip install snscrape

In [103]:
import re
import string
import snscrape.modules.twitter as sntwitter

In [55]:
from datetime import datetime
start_time = datetime.now()

import snscrape.modules.twitter as sntwitter
import pandas
# Creating list to append tweet data to
tweets_longcovid = []

# Using TwitterSearchScraper to scrape data and append tweets to list
for i,tweet in enumerate(sntwitter.TwitterSearchScraper('longcovid since:2021-01-01 until:2021-05-31 lang:en').get_items()):
    if i>50000:
        break
    tweets_longcovid.append([tweet.date, tweet.id, tweet.content, tweet.username])
end_time = datetime.now()
    #Printing the time duration for scraping these tweets

print('Duration: {}'.format(end_time - start_time))

Duration: 0:26:06.550330


In [57]:
# Creating a dataframe from the tweets list above and save it to csv
tweets_longcovid_df = pd.DataFrame(tweets_longcovid, columns=['Datetime', 'Tweet Id', 'Text', 'Username'])
tweets_longcovid_df.to_csv(r'C:\Users\Hafidz\PhDwork\longcovidNLP\tweets_longcovid_df50K.csv', index=False)

In [140]:
# Display first 5 entries from dataframe
tweets_longcovid_df.head()

,Datetime,Tweet Id,Text,Username
0,2021-05-30 23:59:29+00:00,1399153522545942531,"@amyklobuchar @All100Senators Its not over, ev...",bargdaffy161
1,2021-05-30 23:55:37+00:00,1399152549345861647,@PatrickTrottie8 @driusan More people catch it...,zuckerman_l
2,2021-05-30 23:53:36+00:00,1399152043269443589,@Huge761 @scotgov Case numbers mean a lot to t...,HelenSamfat
3,2021-05-30 23:50:06+00:00,1399151160536322056,"Defies belief that a year on &amp; after 150,0...",lottieflying
4,2021-05-30 23:50:04+00:00,1399151151468187649,"@apple_and_I Oh, I can totally relate to the n...",rtofthepossible


In [110]:
tweets_longcovid_df.columns

Index(['Datetime', 'Tweet Id', 'Text', 'Username'], dtype='object')

In [141]:
tweets_longcovid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50001 entries, 0 to 50000
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype              
---  ------    --------------  -----              
 0   Datetime  50001 non-null  datetime64[ns, UTC]
 1   Tweet Id  50001 non-null  int64              
 2   Text      50001 non-null  object             
 3   Username  50001 non-null  object             
dtypes: datetime64[ns, UTC](1), int64(1), object(2)
memory usage: 1.5+ MB


In [112]:
tweets_longcovid_df.shape

(50001, 4)

In [142]:
!{sys.executable} -m pip install tensorflow

In [184]:
!{sys.executable} -m pip install tensorflow
!{sys.executable} -m pip install torch
!{sys.executable} -m pip install transformers
import transformers
import tensorflow as tf
from transformers import BertModel, AdamW, get_linear_schedule_with_warmup
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

ImportError: 
TFBertForSequenceClassification requires the TensorFlow library but it was not found in your environment. Checkout the instructions on the
installation page: https://www.tensorflow.org/install and follow the ones that match your environment.


In [190]:
# Creating TRAIN dataset
#Taking only 10,000 records and creating new dataframe called df1
df50K_train = tweets_longcovid_df.head(10000)
df50K_train.head()

,Datetime,Tweet Id,Text,Username
0,2021-05-30 23:59:29+00:00,1399153522545942531,"@amyklobuchar @All100Senators Its not over, ev...",bargdaffy161
1,2021-05-30 23:55:37+00:00,1399152549345861647,@PatrickTrottie8 @driusan More people catch it...,zuckerman_l
2,2021-05-30 23:53:36+00:00,1399152043269443589,@Huge761 @scotgov Case numbers mean a lot to t...,HelenSamfat
3,2021-05-30 23:50:06+00:00,1399151160536322056,"Defies belief that a year on &amp; after 150,0...",lottieflying
4,2021-05-30 23:50:04+00:00,1399151151468187649,"@apple_and_I Oh, I can totally relate to the n...",rtofthepossible


In [178]:
#Save into csv after applying stopwords
df50K_train.to_csv(r'C:\Users\Hafidz\PhDwork\longcovidNLP\df50K_train_10K.csv', index = False)

In [191]:
!{sys.executable} -m pip install tensorflow
!{sys.executable} -m pip install torch
!{sys.executable} -m pip install transformers
import torch
import transformers
import tensorflow as tf

from transformers import pipeline
#sentiment_classifier = pipeline('sentiment-analysis', model="siebert/sentiment-roberta-large-english")
sentiment_classifier = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


RuntimeError: At least one of TensorFlow 2.0 or PyTorch should be installed. To install TensorFlow 2.0, read the instructions at https://www.tensorflow.org/install/ To install PyTorch, read the instructions at https://pytorch.org/.

In [192]:
df50K_train.head()

,Datetime,Tweet Id,Text,Username
0,2021-05-30 23:59:29+00:00,1399153522545942531,"@amyklobuchar @All100Senators Its not over, ev...",bargdaffy161
1,2021-05-30 23:55:37+00:00,1399152549345861647,@PatrickTrottie8 @driusan More people catch it...,zuckerman_l
2,2021-05-30 23:53:36+00:00,1399152043269443589,@Huge761 @scotgov Case numbers mean a lot to t...,HelenSamfat
3,2021-05-30 23:50:06+00:00,1399151160536322056,"Defies belief that a year on &amp; after 150,0...",lottieflying
4,2021-05-30 23:50:04+00:00,1399151151468187649,"@apple_and_I Oh, I can totally relate to the n...",rtofthepossible


In [209]:
from nltk.parse import CoreNLPParser
result = CoreNLPParser(url='http://localhost:9000')

!{sys.executable} -m pip install pycorenlp
import pycorenlp
from pycorenlp import StanfordCoreNLP
nlp = StanfordCoreNLP('http://localhost:9000')

  Using cached pycorenlp-0.3.0.tar.gz (1.3 kB)
  Created wheel for pycorenlp: filename=pycorenlp-0.3.0-py3-none-any.whl size=2144 sha256=ecb2efc9384fb5b15fbc1fb4cf5f0a307b9591092fa7980f32144136bcb18c72
  Stored in directory: c:\users\hafidz\appdata\local\pip\cache\wheels\83\d8\ad\6b2276343ac605ee47e6beddb28331e96377909e5c816539c3
Successfully built pycorenlp


In [219]:
print(tweets_longcovid_df['Text'][7890])

Had disturbed sleep for months, often I wake due to sounding like a Mongolian throat singer (see YouTube) half choking/gargling. Joked at first but should I worry? especially as the cat looks at me in a WTF/calculating how many meals I'd last it if I was to expire way! #LongCovid


In [211]:
trytext = "My husband is trying an antihistamine diet to see if it helps with any of his long Covid symptoms and I’m on dinner duty and good lord he’s about to eat some boring fish."

In [213]:
import nltk
from nltk.corpus import stopwords
stop = stopwords.words('english')
print(stop)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [217]:
tweets_longcovid_df = tweets_longcovid_df.loc[:,["Datetime","Tweet Id","Text", "Username"]]
# Exclude stopwords 
tweets_longcovid_df['tweet_without_stopwords'] = tweets_longcovid_df['Text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
print(tweets_longcovid_df)

                       Datetime             Tweet Id  \
0     2021-05-30 23:59:29+00:00  1399153522545942531   
1     2021-05-30 23:55:37+00:00  1399152549345861647   
2     2021-05-30 23:53:36+00:00  1399152043269443589   
3     2021-05-30 23:50:06+00:00  1399151160536322056   
4     2021-05-30 23:50:04+00:00  1399151151468187649   
...                         ...                  ...   
49996 2021-04-25 21:28:07+00:00  1386431855616958465   
49997 2021-04-25 21:26:46+00:00  1386431515437920257   
49998 2021-04-25 21:26:05+00:00  1386431341722542081   
49999 2021-04-25 21:25:41+00:00  1386431244049666051   
50000 2021-04-25 21:24:33+00:00  1386430959310958593   

                                                    Text         Username  \
0      @amyklobuchar @All100Senators Its not over, ev...     bargdaffy161   
1      @PatrickTrottie8 @driusan More people catch it...      zuckerman_l   
2      @Huge761 @scotgov Case numbers mean a lot to t...      HelenSamfat   
3      Defies belie